<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/CLAUDE_ORCHESTRATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anthropic -q
!pip install colab-env -q


In [ ]:
import colab_env
colab_env.RELOAD()

In [5]:
import os
import json
import time
import anthropic
from concurrent.futures import ThreadPoolExecutor

# 1. SETUP CLIENT
# Ensure your environment is set with: export CLAUDE3_API_KEY="sk-ant-..."
api_key = os.environ.get("CLAUDE3_API_KEY")
if not api_key:
    raise ValueError("CLAUDE3_API_KEY environment variable not set.")

client = anthropic.Anthropic(api_key=api_key)

# The specific model you requested for this demo
MODEL_NAME = "claude-opus-4-5-20251101"

class Orchestrator:
    def __init__(self):
        self.task_store = {}
        self.results = {}
        self.lock = False # Simplified lock for demo state

    def get_plan(self, goal):
        """Phase 1: The Brain - Decomposes the goal into a DAG."""
        print(f"🧠 Brain: Planning for goal via {MODEL_NAME}...")

        system_prompt = """
        You are a Principal Systems Architect. Decompose the user's goal into a JSON Directed Acyclic Graph (DAG).
        Each task must have:
        - id: string (e.g., 'research_1')
        - description: string
        - dependencies: list of IDs (tasks that must finish first)
        - agent_role: string (e.g., 'security_researcher', 'technical_writer')

        Return ONLY valid JSON. No preamble.
        """

        try:
            message = client.messages.create(
                model=MODEL_NAME,
                max_tokens=2000,
                system=system_prompt,
                messages=[{"role": "user", "content": goal}]
            )

            # Extract content and handle potential markdown wrapping
            raw_text = message.content[0].text.strip()
            if "```json" in raw_text:
                raw_text = raw_text.split("```json")[1].split("```")[0].strip()
            elif "```" in raw_text:
                raw_text = raw_text.split("```")[1].split("```")[0].strip()

            plan = json.loads(raw_text)

            # Map into internal state
            for task in plan['tasks']:
                task['status'] = 'PENDING'
                self.task_store[task['id']] = task
            return plan

        except anthropic.NotFoundError:
            print(f"❌ Model {MODEL_NAME} not found. Check your API tier permissions.")
            raise
        except Exception as e:
            print(f"❌ Planning Error: {e}")
            raise

    def execute_task(self, task_id):
        """Phase 2: The Worker - Executes an individual node."""
        task = self.task_store[task_id]
        print(f"🚀 Dispatching [{task['agent_role']}] for: {task_id}")

        # Build context from previous tasks
        context_data = {dep: self.results.get(dep) for dep in task['dependencies']}

        try:
            response = client.messages.create(
                model=MODEL_NAME,
                max_tokens=1500,
                system=f"You are a specialized {task['agent_role']}. Complete this task using the provided context.",
                messages=[{"role": "user", "content": f"Task: {task['description']}\nContext from prior steps: {context_data}"}]
            )

            self.results[task_id] = response.content[0].text
            self.task_store[task_id]['status'] = 'COMPLETED'
            print(f"✅ Completed: {task_id}")
        except Exception as e:
            print(f"⚠️ Task {task_id} failed: {e}")
            self.task_store[task_id]['status'] = 'FAILED'

    def run_orchestration(self):
        """Phase 3: The Manager - Handles the dependency loop."""
        print("⚙️ Orchestrator: Starting execution loop...")

        with ThreadPoolExecutor(max_workers=3) as executor:
            while any(t['status'] not in ['COMPLETED', 'FAILED'] for t in self.task_store.values()):
                # Identify tasks that are PENDING and have all dependencies MET
                ready_tasks = []
                for tid, t in self.task_store.items():
                    if t['status'] == 'PENDING':
                        if all(self.task_store[dep]['status'] == 'COMPLETED' for dep in t['dependencies']):
                            ready_tasks.append(tid)

                for tid in ready_tasks:
                    self.task_store[tid]['status'] = 'RUNNING'
                    executor.submit(self.execute_task, tid)

                time.sleep(1) # Poll every second for status changes

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    orchestrator = Orchestrator()
    goal = "Create a 3-step technical blog post about Kubernetes security: 1. Research, 2. Write, 3. Review."

    try:
        # Start the 1-hour build process
        orchestrator.get_plan(goal)
        orchestrator.run_orchestration()

        print("\n" + "="*50)
        print("BUILD COMPLETE: PROJECT ARTIFACTS GENERATED")
        print("="*50)
        for tid, content in orchestrator.results.items():
            print(f"\n[NODE: {tid}]\n{content[:400]}...")

    except Exception as e:
        print(f"\nFATAL: Orchestrator halted. {e}")

🧠 Brain: Planning for goal via claude-opus-4-5-20251101...
⚙️ Orchestrator: Starting execution loop...
🚀 Dispatching [security_researcher] for: research_1
✅ Completed: research_1
🚀 Dispatching [technical_writer] for: write_2
✅ Completed: write_2
🚀 Dispatching [technical_reviewer] for: review_3
✅ Completed: review_3

BUILD COMPLETE: PROJECT ARTIFACTS GENERATED

[NODE: research_1]
# Kubernetes Security Research: Comprehensive Analysis

## Executive Summary

This research document compiles authoritative findings on Kubernetes security best practices, common vulnerabilities, RBAC configurations, network policies, pod security standards, and container runtime security. All findings are sourced from official documentation, CVE databases, and recognized security organizations.

...

[NODE: write_2]
# Kubernetes Security Fundamentals: A Practical Guide to Protecting Your Clusters

*Securing your Kubernetes infrastructure with RBAC, Network Policies, and Pod Security Standards*

---

## Introdu